In [9]:
import os
import numpy as np 
import cv2
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from keras.optimizers import Adam
from keras import backend as K
from sklearn.model_selection import train_test_split


def unet(input_size=(256, 256, 1)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return -(2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return dice_coef(y_true, y_pred)

def preprocess_image(image_path, input_size=(512, 512)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image as grayscale
    image = cv2.resize(image, input_size)  # Resize the image to (512, 512)
    image = image / 255.0  # Normalize the pixel values
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def segment_image(model, image):
    mask = model.predict(image)
    return mask

pretrained_weights_path = r"C:/Users/adila/OneDrive/Desktop/cxr_reg_weights.best.hdf5"
image_path = r"C:\Users\adila\OneDrive\Desktop\PREPROCESSING\aheqclahe\5clahe_image.jpg"
output_mask_path = r"C:\Users\adila\OneDrive\Desktop\segmented_mask.jpg"
output_overlayed_path = r"C:\Users\adila\OneDrive\Desktop\overlayed_image.jpg"

pretrained_model = unet(input_size=(512, 512, 1))
learning_rate = 1e-3
batch_size = 64
epochs = 20

pretrained_model.compile(optimizer=Adam(learning_rate=learning_rate), loss=dice_coef_loss, metrics=[dice_coef, 'binary_accuracy'])
pretrained_model.load_weights(pretrained_weights_path) 

preprocessed_image = preprocess_image(image_path)

segmented_mask = segment_image(pretrained_model, preprocessed_image)

# Save the segmented mask image
cv2.imwrite(output_mask_path, segmented_mask[0] * 255)  # Multiply by 255 to convert it back to 0-255 range

# Load the original image
original_image = cv2.imread(image_path)

segmented_mask_resized = cv2.resize(segmented_mask[0], (original_image.shape[1], original_image.shape[0]))
inverted_mask = cv2.bitwise_not(segmented_mask_resized)
inverted_mask = cv2.cvtColor(inverted_mask, cv2.COLOR_GRAY2BGR)

overlayed_image = cv2.addWeighted(original_image, 1, inverted_mask, 1, 0, dtype=cv2.CV_8U)

# Save the overlayed image
cv2.imwrite(output_overlayed_path, overlayed_image)

print("Overlaying black background with lung region and saving complete.")

1/1 [==============================] - 0s 485ms/step
Overlaying black background with lung region and saving complete.
